In [1]:
import os
import numpy as np
from scipy.spatial.transform import Rotation as R
from aist_plusplus.loader import AISTDataset

from tools import preprocessing

In [2]:
anno_dir = "/srv/share/datasets/AIST/aist_plusplus_final/"
subset_size = 20

dataset = AISTDataset(anno_dir)

seq_names = []
seq_names += np.loadtxt(
    os.path.join(anno_dir, "splits/crossmodal_train.txt"), dtype=str
).tolist()
seq_names += np.loadtxt(
    os.path.join(anno_dir, "splits/crossmodal_val.txt"), dtype=str
).tolist()
seq_names += np.loadtxt(
    os.path.join(anno_dir, "splits/crossmodal_test.txt"), dtype=str
).tolist()
ignore_list = np.loadtxt(
    os.path.join(anno_dir, "ignore_list.txt"), dtype=str
).tolist()
seq_names = [name for name in seq_names if name not in ignore_list]
seq_names = seq_names[:subset_size]

In [3]:
seq_names[0]

'gWA_sFM_cAll_d25_mWA4_ch05'

In [4]:
motion = preprocessing.compute_SMPL_motion(seq_names[0], dataset.motion_dir)
motion.shape

(1919, 219)

In [5]:
smpl_poses, smpl_scaling, smpl_trans = AISTDataset.load_motion(dataset.motion_dir, seq_names[0])
smpl_trans /= smpl_scaling
smpl_poses.shape
# 24 rotation vectors. Each is 3D: magnitude is angle, direction is axis, so the vector is an axis-angle rep.

(1919, 72)

In [6]:
def rotation_6d_to_matrix(d6: np.ndarray) -> np.ndarray:
    """
    Converts 6D rotation representation by Zhou et al. [1] to rotation matrix
    using Gram--Schmidt orthogonalization per Section B of [1].
    Args:
        d6: 6D rotation representation, of size (*, 6)
    Returns:
        batch of rotation matrices of size (*, 3, 3)
    [1] Zhou, Y., Barnes, C., Lu, J., Yang, J., & Li, H.
    On the Continuity of Rotation Representations in Neural Networks.
    IEEE Conference on Computer Vision and Pattern Recognition, 2019.
    Retrieved from http://arxiv.org/abs/1812.07035
    """

    a1, a2 = d6[..., :3], d6[..., 3:]
    b1 = a1 / np.linalg.norm(a1, axis=-1, keepdims=True)
    b2 = a2 - (b1 * a2).sum(-1, keepdims=True) * b1
    b2 = b2 / np.linalg.norm(b2, axis=-1, keepdims=True)
    b3 = np.cross(b1, b2, axis=-1)
    return np.stack((b1, b2, b3), axis=-2)


def matrix_to_rotation_6d(matrix: np.ndarray) -> np.ndarray:
    """
    Converts rotation matrices to 6D rotation representation by Zhou et al. [1]
    by dropping the last row. Note that 6D representation is not unique.
    Args:
        matrix: batch of rotation matrices of size (*, 3, 3)
    Returns:
        6D rotation representation, of size (*, 6)
    [1] Zhou, Y., Barnes, C., Lu, J., Yang, J., & Li, H.
    On the Continuity of Rotation Representations in Neural Networks.
    IEEE Conference on Computer Vision and Pattern Recognition, 2019.
    Retrieved from http://arxiv.org/abs/1812.07035
    """
    return np.copy(matrix[..., :2, :]).reshape(*matrix.shape[:-2], 6)

In [7]:
motion_reshaped = motion[:,3:].reshape((motion.shape[0], -1, 3, 3))
motion_6D = matrix_to_rotation_6d(motion_reshaped)
motion_rotmat_recovered = rotation_6d_to_matrix(motion_6D)
print(motion_6D.shape, motion_rotmat_recovered.shape, motion_reshaped.shape, np.allclose(motion_reshaped, motion_rotmat_recovered))
motion_reshaped[0,0,...], motion_rotmat_recovered[0,0,...]

(1919, 24, 6) (1919, 24, 3, 3) (1919, 24, 3, 3) True


(array([[ 0.84896038,  0.02461495, -0.52788292],
        [-0.07640416,  0.99413637, -0.07651979],
        [ 0.52290408,  0.10529472,  0.84586308]]),
 array([[ 0.84896038,  0.02461495, -0.52788292],
        [-0.07640416,  0.99413637, -0.07651979],
        [ 0.52290408,  0.10529472,  0.84586308]]))

In [8]:
def new_compute_SMPL_motion(seq_name: str, motion_dir: str) -> np.ndarray:
    smpl_poses, smpl_scaling, smpl_trans = AISTDataset.load_motion(motion_dir, seq_name)
    smpl_trans /= smpl_scaling

    smpl_poses = R.from_rotvec(smpl_poses.reshape((-1, 3))).as_matrix().reshape((smpl_poses.shape[0], -1, 3, 3))
    smpl_poses = matrix_to_rotation_6d(smpl_poses).reshape((smpl_poses.shape[0], -1))

    smpl_motion = np.concatenate([smpl_trans, smpl_poses], axis=-1)
    return smpl_motion

In [10]:
new_motion = new_compute_SMPL_motion(seq_names[0], dataset.motion_dir)
new_motion.shape

(1919, 147)

In [11]:
def new_to_old_motion(new_motion: np.ndarray) -> np.ndarray:
    trans, rot_6d = new_motion[:, :3], new_motion[:, 3:]
    rotmats = rotation_6d_to_matrix(rot_6d.reshape((rot_6d.shape[0], -1, 6)))
    rotmats = rotmats.reshape((rot_6d.shape[0], -1))
    return np.concatenate([trans, rotmats], axis=-1)

In [12]:
old_motion_recovered = new_to_old_motion(new_motion)
old_motion_recovered.shape

(1919, 219)

In [18]:
# Interpolation
i1, i2 = 0, 1
m1, m2 = old_motion_recovered[0, 3+9*i1:12+9*i1].reshape((3,3)), old_motion_recovered[0, 3+9*i2:12+9*i2].reshape((3,3))
m1, m2

(array([[ 0.84896038,  0.02461495, -0.52788292],
        [-0.07640416,  0.99413637, -0.07651979],
        [ 0.52290408,  0.10529472,  0.84586308]]),
 array([[ 0.95591427,  0.27488122, -0.10328714],
        [-0.29360241,  0.8886426 , -0.35229526],
        [-0.00505399,  0.36708942,  0.93017193]]))

In [20]:
m_half = 0.5 * (m1 + m2)
m_half

array([[ 0.90243732,  0.14974808, -0.31558503],
       [-0.18500328,  0.94138949, -0.21440753],
       [ 0.25892504,  0.23619207,  0.8880175 ]])

In [21]:
n1, n2 = new_motion[0, 3+6*i1:9+6*i1], new_motion[0, 3+6*i2:9+6*i2]
n1, n2

(array([ 0.84896038,  0.02461495, -0.52788292, -0.07640416,  0.99413637,
        -0.07651979]),
 array([ 0.95591427,  0.27488122, -0.10328714, -0.29360241,  0.8886426 ,
        -0.35229526]))

In [22]:
n_half = 0.5 * (n1 + n2)
n_half

array([ 0.90243732,  0.14974808, -0.31558503, -0.18500328,  0.94138949,
       -0.21440753])

In [23]:
rotation_6d_to_matrix(n_half[None, ...])

array([[[ 0.9325748 ,  0.15474901, -0.3261242 ],
        [-0.2292721 ,  0.95174307, -0.20400841],
        [ 0.27881634,  0.26502428,  0.92305123]]])

In [27]:
R.from_matrix(rotation_6d_to_matrix(n_half[None, ...])).as_matrix() - rotation_6d_to_matrix(n_half[None, ...])

array([[[-1.11022302e-16, -2.77555756e-17,  5.55111512e-17],
        [ 5.55111512e-17, -2.22044605e-16,  5.55111512e-17],
        [-5.55111512e-17,  0.00000000e+00, -2.22044605e-16]]])

In [28]:
R.from_matrix(m_half).as_matrix() - m_half

array([[ 0.03688114, -0.01381917,  0.00061811],
       [-0.02198716,  0.01536287,  0.01000625],
       [ 0.01463626,  0.02100096,  0.03881364]])

In [30]:
from tools import visualize

In [34]:
visualize.get_closest_rotmat(m_half[None, ...]), R.from_matrix(m_half).as_matrix(), rotation_6d_to_matrix(n_half[None, ...])

(array([[[ 0.93900494,  0.13519189, -0.31621651],
         [-0.20635196,  0.95706335, -0.20358931],
         [ 0.27511561,  0.25642326,  0.92658432]]]),
 array([[ 0.93931846,  0.13592891, -0.31496692],
        [-0.20699044,  0.95675236, -0.20440127],
        [ 0.2735613 ,  0.25719303,  0.92683114]]),
 array([[[ 0.9325748 ,  0.15474901, -0.3261242 ],
         [-0.2292721 ,  0.95174307, -0.20400841],
         [ 0.27881634,  0.26502428,  0.92305123]]]))

In [36]:
rotation_6d_to_matrix(n_half[None, ...]) - visualize.get_closest_rotmat(rotation_6d_to_matrix(n_half[None, ...]))

array([[[ 0.00000000e+00, -5.55111512e-17, -5.55111512e-17],
        [ 0.00000000e+00,  1.11022302e-16, -8.32667268e-17],
        [ 0.00000000e+00,  5.55111512e-17,  1.11022302e-16]]])